In [ ]:
import pandas as pd
import sys
import os

In [ ]:
# regulate the base path for different environments
if (sys.platform.startswith("linux")):
    pathNav = "/"
else:
    pathNav = "\\"

idx = os.path.abspath('').split(pathNav).index("roi-prediction") + 1
base_path = pathNav.join(os.path.abspath('').split(pathNav)[:idx])

In [ ]:
states_list = [
    "United States *",
    "Alabama",
    "Alaska",
    "Arizona",
    "Arkansas",
    "California",
    "Colorado",
    "Connecticut",
    "Delaware",
    "Florida",
    "Georgia",
    "Hawaii",
    "Idaho",
    "Illinois",
    "Indiana",
    "Iowa",
    "Kansas",
    "Kentucky",
    "Louisiana",
    "Maine",
    "Maryland",
    "Massachusetts",
    "Michigan",
    "Minnesota",
    "Mississippi",
    "Missouri",
    "Montana",
    "Nebraska",
    "Nevada",
    "New Hampshire",
    "New Jersey",
    "New Mexico",
    "New York",
    "North Carolina",
    "North Dakota",
    "Ohio",
    "Oklahoma",
    "Oregon",
    "Pennsylvania",
    "Rhode Island",
    "South Carolina",
    "South Dakota",
    "Tennessee",
    "Texas",
    "Utah",
    "Vermont",
    "Virginia",
    "Washington",
    "West Virginia",
    "Wisconsin",
    "Wyoming"
  ]

years_list = [
    "2001",
    "2002",
    "2003",
    "2004",
    "2005", 
    "2006",
    "2007",
    "2008",
    "2009",
    "2010",
    "2011",
    "2012"
  ]

In [ ]:
# function to reset the index of any dataframe if is not in sequence order
def reset_sub_df_index(df, have_index_column=None):
    new_df = df.copy()
    if (have_index_column != None and have_index_column == True):
        new_df.drop("index", axis=1, inplace=True)
        
    new_df = new_df.reset_index()
    new_df.drop("index", axis=1, inplace=True)
    new_df = new_df.reset_index()
    return new_df

In [ ]:
def read_dataframe_from_folder(parent_path, file_name):
    return pd.read_csv(base_path + pathNav + parent_path + pathNav + file_name)

In [ ]:
def write_dataframe_to_folder(df, file_name, destination_path):
    if (not os.path.exists(base_path + pathNav + destination_path)):
        os.makedirs(base_path + pathNav + destination_path)

    df.to_csv(base_path + pathNav + destination_path + pathNav + file_name, sep=',', index=False, encoding='utf-8', header=True)

In [ ]:
# filter out rows that has unwanted values in a column
def filter_unwanted_value(df, column, list_values):
    df = df[~df[column].isin(list_values)]
    return df

In [ ]:
# join columns into dataframe
def left_join_dataframe(df_1, df_2, columns):
    df_1 = pd.merge(df_1, df_2, on=columns)

    return df_1

In [ ]:
# sort dataframe
def sort_dataframe_on_column(df, target_columns, groupby_column=None):
    if (groupby_column == None):
        df = df.sort_values(by=target_columns, ascending=True).apply(lambda a: a[:]).reset_index()
    else:    
        df = df.sort_values(by=target_columns, ascending=True).groupby(groupby_column).apply(lambda a: a[:], include_groups=False).reset_index()

    df.drop("index", axis=1, inplace=True)
    df = df.reset_index()
    df.drop("level_1", axis=1, inplace=True)
    return df

In [ ]:
# filter dataframe based on columns list and its data
def filter_dataframe_on_column(df, target_columns, target_values):
    mask = df[target_columns].apply(lambda x: True if tuple(x.values) == tuple(target_values) else False, axis=1)
    return df[mask]

In [ ]:
# filter dataframe based on columns list and its data but 
def filter_dataframe_exclude_rows(df, target_columns, target_values):
    mask = df[target_columns].apply(lambda x: tuple(x.values) != tuple(target_values), axis=1)
    return df[mask]

In [ ]:
# function to get rid of date-formatted columns with target date to cut
def filter_columns_with_date_name_format(df, date_format="%Y-%m-%d", start="2013-01-01", end="2023-12-31"):
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)

    filtered_cols = []

    for col in df.columns:
        try:
            col_date = pd.to_datetime(col, format=date_format)
            if start_date <= col_date <= end_date:
                filtered_cols.append(col)
        except (ValueError, TypeError):
            filtered_cols.append(col)

    return df[filtered_cols]

In [ ]:
# Combine two columns into a new one
def combine_columns(df, first_column, second_column, new_column_name):
    combined_values = df[first_column].astype(str).apply(lambda x: x.zfill(2)) + df[second_column].astype(str).apply(lambda x: x.zfill(3))
    df[new_column_name] = combined_values

    return df

In [ ]:
def pad_column_with_zeros(df, target_column, desired_length):
    df[target_column] = df[target_column].astype(str).str.zfill(desired_length)
    return df

In [ ]:
# copy the dataframe with kept target columns
def copy_dataframe_with_desired_columns(df, target_columns):
    df_copy = df[target_columns].copy()
    df_copy = df_copy.drop_duplicates()

    return df_copy

In [ ]:
# copy the dataframe without target columns
def copy_dataframe_without_target_columns(df, target_columns):
    df_copy = df.drop(columns=target_columns).copy()
    df_copy = df_copy.drop_duplicates()

    return df_copy

In [ ]:
# function to rename some certain columns
def rename_columns(df, target_columns, new_values):
    rename_dict = dict(zip(target_columns, new_values))

    return df.rename(columns=rename_dict)

In [ ]:
# function to split a value of a string value into left and right based on separator
def split_string_column(df, source_column, value_separator, new_column, is_left=True):
    if is_left:
        df[new_column] = df[source_column].str.split(value_separator).str[0]
    else:
        df[new_column] = df[source_column].str.split(value_separator).str[-1]

    return df

In [ ]:
# read in all relevant datasets unfiltered
    # time series
df_unfiltered_zhvi = read_dataframe_from_folder("datasets", "county_zhvi.csv")
df_unfiltered_county_gdp = read_dataframe_from_folder("datasets", "CAGDP2__ALL_AREAS_2001_2023.csv")
df_unfiltered_unemployment_rate = read_dataframe_from_folder("datasets", "county_unemployment_2013_2023_combined.csv")

    # static
df_unfiltered_population = read_dataframe_from_folder("datasets", "demographic_5_years_est_population_2023.csv")
df_unfiltered_median_income = read_dataframe_from_folder("datasets", "median_income_5_years_est_2023.csv")
df_unfiltered_property_tax_rate = read_dataframe_from_folder("datasets", "property_tax_rate.csv")
df_unfiltered_mobile_broadband = read_dataframe_from_folder("datasets", "mobile_broadband.csv")

    # mapping
df_unfiltered_FIPS = read_dataframe_from_folder("datasets", "FIPS_code.csv")

In [ ]:
# filter on desired columns
df_unfiltered_zhvi = copy_dataframe_without_target_columns(df_unfiltered_zhvi, ["RegionID", "SizeRank", "RegionType", "StateName", "State", "Metro"])
df_unfiltered_zhvi = combine_columns(df_unfiltered_zhvi, "StateCodeFIPS", "MunicipalCodeFIPS", "geoID")
df_unfiltered_zhvi = copy_dataframe_without_target_columns(df_unfiltered_zhvi, ["StateCodeFIPS", "MunicipalCodeFIPS"])

df_unfiltered_FIPS = combine_columns(df_unfiltered_FIPS, "StateFIPS", "MunicipalityFIPS", "geoID")
df_unfiltered_FIPS = copy_dataframe_without_target_columns(df_unfiltered_FIPS, ["StateFIPS", "MunicipalityFIPS", "FIPS"])

df_unfiltered_county_gdp = copy_dataframe_without_target_columns(df_unfiltered_county_gdp, ["Region", "TableName", "LineCode", "IndustryClassification", "Unit"])
df_unfiltered_county_gdp = rename_columns(df_unfiltered_county_gdp, ["GeoFIPS", "GeoName"], ["geoID", "name"])
df_unfiltered_county_gdp = copy_dataframe_without_target_columns(df_unfiltered_county_gdp, years_list)

df_unfiltered_unemployment_rate = copy_dataframe_without_target_columns(df_unfiltered_unemployment_rate, ["series_id"])
df_unfiltered_unemployment_rate = rename_columns(df_unfiltered_unemployment_rate, ["area_name"], ["Name"])

df_unfiltered_population = copy_dataframe_with_desired_columns(df_unfiltered_population, ["GEO_ID", "NAME", "DP05_0001E"])
df_unfiltered_population = split_string_column(df_unfiltered_population, "GEO_ID", "S", "geoID", False)
df_unfiltered_population = copy_dataframe_without_target_columns(df_unfiltered_population, ["GEO_ID"])
df_unfiltered_population = rename_columns(df_unfiltered_population, ["NAME", "DP05_0001E"], ["name", "estimate"])

df_unfiltered_median_income = copy_dataframe_with_desired_columns(df_unfiltered_median_income, ["GEO_ID", "NAME", "S1903_C01_001E"])
df_unfiltered_median_income = split_string_column(df_unfiltered_median_income, "GEO_ID", "S", "geoID", False)
df_unfiltered_median_income = copy_dataframe_without_target_columns(df_unfiltered_median_income, ["GEO_ID"])
df_unfiltered_median_income = rename_columns(df_unfiltered_median_income, ["NAME", "S1903_C01_001E"], ["name", "estimate"])

df_unfiltered_property_tax_rate = copy_dataframe_with_desired_columns(df_unfiltered_property_tax_rate, ["geoid", "name", "effective_prop_tax_rate_23"])
df_unfiltered_property_tax_rate = rename_columns(df_unfiltered_property_tax_rate, ["geoid", "effective_prop_tax_rate_23"], ["geoID", "tax_rate"])

df_unfiltered_mobile_broadband = copy_dataframe_without_target_columns(df_unfiltered_mobile_broadband, ["mobilebb_3g_area_iv_pct", "mobilebb_4g_area_iv_pct", "mobilebb_5g_spd1_area_iv_pct", "mobilebb_5g_spd2_area_iv_pct"])
df_unfiltered_mobile_broadband = rename_columns(df_unfiltered_mobile_broadband, ["geography_desc", "geography_id", "mobilebb_3g_area_st_pct", "mobilebb_4g_area_st_pct", "mobilebb_5g_spd1_area_st_pct", "mobilebb_5g_spd2_area_st_pct"], ["name", "geoID", "3g_st_pct", "4g_st_pct", "5g_spd1_st_pct", "5g_spd2_st_pct"])

In [ ]:
# filter on desired rows
df_unfiltered_county_gdp = filter_unwanted_value(df_unfiltered_county_gdp, "name", states_list)
df_unfiltered_county_gdp = filter_dataframe_on_column(df_unfiltered_county_gdp, ["Description"], ["All industry total "])
df_unfiltered_population = filter_unwanted_value(df_unfiltered_population, "name", ["Geographic Area Name", "United States"])
df_unfiltered_median_income = filter_unwanted_value(df_unfiltered_median_income, "name", ["Geographic Area Name", "United States"])
df_unfiltered_unemployment_rate = left_join_dataframe(df_unfiltered_unemployment_rate, df_unfiltered_FIPS, ["Name"])
df_unfiltered_mobile_broadband = filter_dataframe_on_column(df_unfiltered_mobile_broadband, ["area_data_type", "geography_type"], ["Total", "County"])

In [ ]:
# last filtering
df_county_gdp = copy_dataframe_without_target_columns(df_unfiltered_county_gdp, ["Description"])
df_population = df_unfiltered_population
df_zhvi = filter_columns_with_date_name_format(df_unfiltered_zhvi)
df_FIPS = pad_column_with_zeros(df_unfiltered_FIPS, "geoID", 5)
df_median_income = df_unfiltered_median_income
df_unemployment_rate = copy_dataframe_without_target_columns(df_unfiltered_unemployment_rate, ["StateName"])
df_property_tax_rate = pad_column_with_zeros(df_unfiltered_property_tax_rate, "geoID", 5)
df_mobile_broadband = copy_dataframe_without_target_columns(df_unfiltered_mobile_broadband, ["area_data_type", "geography_type"])

In [ ]:
# write to files
write_dataframe_to_folder(df_county_gdp, "county_gdp.csv", "datasets_filtered")
write_dataframe_to_folder(df_population, "demographic.csv", "datasets_filtered")
write_dataframe_to_folder(df_zhvi, "zhvi.csv", "datasets_filtered")
write_dataframe_to_folder(df_FIPS, "FIPS_code.csv", "datasets_filtered")
write_dataframe_to_folder(df_median_income, "median_income.csv", "datasets_filtered")
write_dataframe_to_folder(df_unemployment_rate, "unemployment.csv", "datasets_filtered")
write_dataframe_to_folder(df_property_tax_rate, "property_tax.csv", "datasets_filtered")
write_dataframe_to_folder(df_mobile_broadband, "mobile_broadband.csv", "datasets_filtered")